In [10]:
import requests
import pandas as pd
from IPython.display import display
# Base URL of your API
API = 'http://localhost:8000'

In [ ]:
# upload a document

# file ref is the absolut path to the file

resp = requests.post(
    f"{API}/upload-document",
    json={"file_ref": "/Users/antonwiklund/ai-task/backend/data/Bellring Brands/txt/BellringBrandsInc_20190920_S-1_EX-10.12_11817081_EX-10.12_Manufacturing Agreement3.txt"}
)
resp.raise_for_status()
upload_data = resp.json()
document_id_1 = upload_data['document_id']
print(f"Uploaded document, got document_id: {document_id_1}, message {upload_data['message']}")

In [ ]:
# upload another document

resp = requests.post(
    f"{API}/upload-document",
    json={"file_ref": "/Users/antonwiklund/ai-task/backend/data/AzulSa/txt/AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement2.txt"}
)
resp.raise_for_status()
upload_data = resp.json()
document_id_2 = upload_data['document_id']
print(f"Uploaded document, got document_id: {document_id_2}, message {upload_data['message']}")

In [ ]:
# create columns

column_payload = {
    "label": "Agreement date",
    "prompt": "What is the date when the agreement went into force?",
    "format": "date"
}
resp = requests.post(f"{API}/columns", json=column_payload)
resp.raise_for_status()
column_id = resp.json()['column_id']
print(f"Created column, got column_id: {column_id}")

In [ ]:
# create a row
row_payload = {
    "document_ids": [document_id_1, document_id_2]
}
resp = requests.post(f"{API}/rows", json=row_payload)
resp.raise_for_status()
row_id = resp.json()['row_id']
print(f"Created row, got row_id: {row_id}")

In [ ]:
# batch call to /answer endpoint and store result
batch_payload = {
    "items": [
        {"row_id": row_id, "column_id": column_id}
    ]
}
resp = requests.post(f"{API}/answer", json=batch_payload)
resp.raise_for_status()
answer_data_for_table = resp.json()
print("Batch Answer Response:", answer_data_for_table)

In [ ]:
column_label_val = 'Unknown Column' # Default value
prompt_val = 'N/A'
format_val = 'N/A'

if 'column_payload' in locals() and isinstance(column_payload, dict):
    column_label_val = column_payload.get('label', 'Unknown Column')
    prompt_val = column_payload.get('prompt', 'N/A')
    format_val = column_payload.get('format', 'N/A')

cell_value_val = "N/A (Run previous cell or no answer found)" # Default value
if 'answer_data_for_table' in locals() and isinstance(answer_data_for_table, dict) and answer_data_for_table.get('results'):
    if len(answer_data_for_table['results']) > 0:
        answer_item = answer_data_for_table['results'][0] # Demo sends one item
        cell_value_val = answer_item['answer']

row_id_val = str(row_id) if 'row_id' in locals() else 'N/A'

if 'row_payload' in locals() and isinstance(row_payload, dict) and 'document_ids' in row_payload:
    actual_document_ids_for_row = row_payload.get('document_ids', [])
    document_id_val = ', '.join(map(str, actual_document_ids_for_row)) if actual_document_ids_for_row else 'N/A'
else:
    document_id_val = 'N/A (row_payload not available or malformed)'

table_display_data = {
    'Row ID': [row_id_val],
    'Document ID(s)': [document_id_val],
    column_label_val: [cell_value_val],
    'Prompt': [prompt_val],
    'Format': [format_val]
}

# Reordering columns for better readability
df_result_table = pd.DataFrame(table_display_data)
desired_order = ['Row ID', 'Document ID(s)', 'Prompt', 'Format', column_label_val]
# Ensure all desired columns are present, add if missing (e.g. if column_label_val is new)
for col in desired_order:
    if col not in df_result_table.columns:
        df_result_table[col] = 'N/A' # or some other default
df_result_table = df_result_table[desired_order]


print("Current Processed Table:")
display(df_result_table)